In [1]:
print("check")

check


In [1]:
!pip install opencv-python numpy gradio

In [3]:
import cv2
import os
import numpy as np

def extract_optical_flows(directory):
    optical_flows = {}
    for file in os.listdir(directory):
        if file.endswith(".mp4"):
            video_path = os.path.join(directory, file)
            print(f"Extracting optical flows for {video_path}...")
            cap = cv2.VideoCapture(video_path)
            ret, frame1 = cap.read()
            prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
            hsv = np.zeros_like(frame1)
            hsv[..., 1] = 255

            flows = []
            frame_idx = 0
            while True:
                ret, frame2 = cap.read()
                if not ret:
                    break
                next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
                flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
                flows.append(flow)
                
                # Save the flow to a file
                flow_filename = f"{os.path.splitext(file)[0]}_flow_{frame_idx}.npy"
                np.save(os.path.join(directory, flow_filename), flow)
                
                prvs = next
                frame_idx += 1

            cap.release()
            #optical_flows[file] = flows

    return optical_flows

# Extract optical flows for each mp4 in the directory
optical_flows = extract_optical_flows(".")

Extracting optical flows for .\curry23.mp4...
Extracting optical flows for .\grant23.mp4...
Extracting optical flows for .\green23.mp4...
Extracting optical flows for .\jokerfloats.mp4...
Extracting optical flows for .\murray22.mp4...
Extracting optical flows for .\thetruth.mp4...


In [ ]:
import glob

def load_optical_flows(directory):
    flow_files = glob.glob(os.path.join(directory, "*_flow_*.npy"))
    flows = {}
    for file in flow_files:
        video_name = os.path.basename(file).split('_flow_')[0]
        frame_idx = int(os.path.basename(file).split('_flow_')[1].split('.npy')[0])
        if video_name not in flows:
            flows[video_name] = []
        flows[video_name].append((frame_idx, np.load(file)))
    return flows

def find_most_similar_flows(flows):
    min_diff = float('inf')
    best_pair = None
    for video1, frames1 in flows.items():
        for video2, frames2 in flows.items():
            if video1 != video2:
                for idx1, flow1 in frames1:
                    for idx2, flow2 in frames2:
                        diff = np.sum((flow1 - flow2) ** 2)
                        if diff < min_diff:
                            min_diff = diff
                            best_pair = (video1, idx1, video2, idx2)
    return best_pair

def create_transition_video(video1_path, video2_path, idx1, idx2, output_path):
    cap1 = cv2.VideoCapture(video1_path)
    cap2 = cv2.VideoCapture(video2_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, 30.0, (int(cap1.get(3)), int(cap1.get(4))))

    for i in range(idx1):
        ret, frame = cap1.read()
        if not ret:
            break
        out.write(frame)

    for i in range(idx2, int(cap2.get(cv2.CAP_PROP_FRAME_COUNT))):
        ret, frame = cap2.read()
        if not ret:
            break
        out.write(frame)

    cap1.release()
    cap2.release()
    out.release()

# Load optical flows from .npy files
flows = load_optical_flows(".")

# Find the most similar optical flows
video1, idx1, video2, idx2 = find_most_similar_flows(flows)

# Create a transition video
create_transition_video(f"{video1}.mp4", f"{video2}.mp4", idx1, idx2, "transition_video.mp4")

In [ ]:
import gradio as gr

def play_video():
    return "transition_video.mp4"

iface = gr.Interface(fn=play_video, inputs=[], outputs=gr.Video())

iface.launch()